# Analyzing Data and Interpreting Images with OpenAI's o1 Reasoning Model vs. GPT

## Introduction
OpenAI's o1 reasoning model is designed for complex problem-solving, data analysis, and image interpretation by simulating a multi-step thought process before generating responses. Unlike traditional GPT models, which produce output in a single pass, reasoning models use internal **reasoning tokens** to explore multiple approaches before finalizing an answer.
<p align="center">
    <img src="https://cdn.openai.com/API/images/guides/reasoning_tokens.png" alt="Reasoning Tokens" width="600">
</p>  

*Source: [OpenAI Reasoning Models Guide](https://platform.openai.com/docs/guides/reasoning)*

**Key Differences: o1 Reasoning Model vs. GPT**
- Multi-step reasoning: o1 evaluates different solutions before selecting the best response.
- Deeper analytical capabilities: Optimized for complex data interpretation tasks.
- Context-aware image analysis: Provides more structured and insightful image descriptions.
- Reasoning Effort Control: Users can adjust the depth of reasoning (`low`, `medium`, `high`).


For more details, refer to the [OpenAI Reasoning Models Guide](https://platform.openai.com/docs/guides/reasoning).


## Purchase and Store API Key

You need to **purchase** your [OpenAI](https://openai.com/) API key and store it securely, such as in **AWS Secrets Manager**.

- **Key Name:** `api_key`  
- **Key Value:** `<your OpenAI API key>`  
- **Secret Name:** `openai`  

## Install Python Libraries

- **openai**: Used to call `o1` and `GPT` models for data analysis and image interpretation.

In [1]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

The following libraries are used in this notebook:

- **boto3**: AWS SDK for Python, used to interact with AWS services.
- **json**: Standard Python library for handling JSON data.
- **IPython.display**: Provides tools to display images, Markdown content, and other rich media in Jupyter Notebook.
- **openai**: Used to call `o1` and `GPT` models for data analysis and image interpretation.
- **pandas**: A powerful library for data manipulation and analysis.
- **pprint**: Pretty prints data structures for better readability.

In [2]:
import boto3
import json
from IPython.display import display, Image, Markdown
from openai import OpenAI
import pandas as pd
from pprint import pprint

## Retrieve API Keys Securely from AWS Secrets Manager

The following function, `get_secret()`, retrieves a secret from **AWS Secrets Manager**. This is a secure way to store and access sensitive credentials, such as API keys, without hardcoding them into the script

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

The following code initializes the OpenAI client using a securely stored API key retrieved from AWS Secrets Manager.

In [4]:
client = OpenAI(api_key= get_secret('openai')['api_key'])

## Load and Analyze the Diamonds Dataset

This notebook uses the **diamonds dataset ([diamonds.csv](https://github.com/lbsocial/data-analysis-with-generative-ai/blob/main/diamonds.csv))**, which contains detailed attributes of diamonds, including weight, color, clarity, and price.

One interesting pattern in the dataset is that **diamonds with "IF" (Internally Flawless) clarity tend to have the lowest average price** compared to other clarity grades. This observation is counterintuitive, as one might expect the highest-clarity diamonds to be the most expensive.

In [5]:
df = pd.read_csv('diamonds.csv')
data_json = df.to_json(orient="records")
df.head()

,IDNO,WEIGHT,COLOR,CLARITY,RATER,PRICE
0,1,0.30,D,VS2,GIA,1302
1,2,0.30,E,VS1,GIA,1510
2,3,0.30,G,VVS1,GIA,1510
3,4,0.30,G,VS1,GIA,1260
4,5,0.31,D,VS1,GIA,1641


## Generate Data Analysis Prompt for OpenAI Model

To investigate why diamonds with **IF (Internally Flawless) clarity** have the **lowest average price**, we generate a structured prompt for the OpenAI model. The model will analyze the dataset and generate insights, including **Python code for visualizations**.


In [6]:
data_prompt = f"Analyze the provided data and determine why diamonds with IF clarity have the lowest average price. Provide Python-generated charts to support your conclusion. Data: {data_json}"
# print(prompt)

## Define a Function to Get Assistance from OpenAI GPT-4o

The following function, `openai_gpt_help()`, sends a prompt to OpenAI's **GPT-4o model** and returns a response. It also prints the number of tokens used in the request.

In [7]:
def openai_gpt_help(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature = 0
    )
    token_usage = response.usage
    
    pprint(f"Tokens used: {token_usage}")

    return response.choices[0].message.content

In [8]:
gpt_result = openai_gpt_help(prompt=data_prompt)

('Tokens used: CompletionUsage(completion_tokens=794, prompt_tokens=10574, '
 'total_tokens=11368, '
 'completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, '
 'audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), '
 'prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))')


In [9]:
display(Markdown(gpt_result))

To analyze why diamonds with IF (Internally Flawless) clarity have the lowest average price, we need to examine the dataset and compare the prices of diamonds with different clarity grades. We will use Python to perform this analysis and generate charts to visualize the data.

First, let's load the data and perform some basic analysis:

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data into a DataFrame
data = [
    {"IDNO": 1, "WEIGHT": 0.3, "COLOR": "D", "CLARITY": "VS2", "RATER": "GIA", "PRICE": 1302},
    {"IDNO": 2, "WEIGHT": 0.3, "COLOR": "E", "CLARITY": "VS1", "RATER": "GIA", "PRICE": 1510},
    # ... (rest of the data)
    {"IDNO": 308, "WEIGHT": 1.09, "COLOR": "I", "CLARITY": "VVS2", "RATER": "HRD", "PRICE": 9107}
]

df = pd.DataFrame(data)

# Calculate the average price for each clarity grade
average_price_by_clarity = df.groupby('CLARITY')['PRICE'].mean().sort_values()

# Plot the average price by clarity
plt.figure(figsize=(10, 6))
sns.barplot(x=average_price_by_clarity.index, y=average_price_by_clarity.values)
plt.title('Average Price by Clarity')
plt.xlabel('Clarity')
plt.ylabel('Average Price')
plt.show()
```

### Analysis

1. **Average Price by Clarity**: The bar plot shows the average price for each clarity grade. If IF clarity diamonds have the lowest average price, it could be due to several factors:
   - **Weight Distribution**: IF diamonds in the dataset might be of lower carat weight, which significantly affects the price.
   - **Color and Rater Influence**: The color and the rater (GIA, IGI, HRD) might also influence the price. IF diamonds might be paired with less desirable colors or raters that typically have lower prices.
   - **Sample Size and Distribution**: The number of IF diamonds in the dataset might be small, and their distribution across other factors like weight and color might skew the average price.

2. **Weight and Color Influence**: Let's further analyze the weight and color distribution for IF diamonds:

```python
# Filter IF diamonds
if_diamonds = df[df['CLARITY'] == 'IF']

# Plot weight distribution for IF diamonds
plt.figure(figsize=(10, 6))
sns.histplot(if_diamonds['WEIGHT'], bins=10, kde=True)
plt.title('Weight Distribution for IF Diamonds')
plt.xlabel('Weight (Carats)')
plt.ylabel('Frequency')
plt.show()

# Plot color distribution for IF diamonds
plt.figure(figsize=(10, 6))
sns.countplot(x='COLOR', data=if_diamonds)
plt.title('Color Distribution for IF Diamonds')
plt.xlabel('Color')
plt.ylabel('Count')
plt.show()
```

### Conclusion

- **Weight Distribution**: If the weight distribution plot shows that IF diamonds are generally of lower carat weight, this could explain the lower average price.
- **Color Distribution**: If the color distribution plot shows that IF diamonds are predominantly of lower color grades (e.g., H, I), this could also contribute to the lower average price.
- **Rater Influence**: If IF diamonds are mostly rated by a rater that typically has lower prices, this could further explain the lower average price.

By examining these factors, we can better understand why IF clarity diamonds have the lowest average price in this dataset.

## Define a Function to Get Assistance from OpenAI o1 Model  

The following function, `openai_o_help()`, sends a prompt to OpenAI's **o1 reasoning model** and returns a response.  

### Key Differences Between o1 and GPT Models:
- **Reasoning Effort**: The o1 model allows users to control reasoning depth using `reasoning_effort` (`low`, `medium`, `high`).  
- **No Temperature Parameter**: Unlike GPT models, **o1 does not support `temperature`**.  
- **Developer Messages Replace System Messages**:  
  - Starting with `o1-2024-12-17`, **developer messages** replace **system messages** to align with chain-of-command behavior.  

### Best Practices for Prompting o1  
- **Keep prompts simple and direct.**  
- **Avoid chain-of-thought prompts.** o1 reasons internally, so step-by-step instructions aren't needed.  
- **Use delimiters for clarity.** Use Markdown, XML tags, or section titles.  
- **Try zero-shot first.** If needed, add few-shot examples that closely match your goal.  
- **Be explicit.** Clearly define success criteria and constraints.  
- **Markdown is disabled by default.** To enable, start with `"Formatting re-enabled"`.  

Source: [OpenAI Reasoning Models Best Practices Guide](https://platform.openai.com/docs/guides/reasoning-best-practices).  


In [10]:
def openai_o_help(prompt):
    messages = [ {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model='o1',
        reasoning_effort="high", # low, medium or high
        messages=messages,

    )
    token_usage = response.usage
    
    pprint(f"Tokens used: {token_usage}")

    return response.choices[0].message.content

In [11]:
o1_result = openai_o_help(prompt=data_prompt)

('Tokens used: CompletionUsage(completion_tokens=3753, prompt_tokens=10573, '
 'total_tokens=14326, '
 'completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, '
 'audio_tokens=0, reasoning_tokens=2560, rejected_prediction_tokens=0), '
 'prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))')


In [12]:
print(o1_result)

Below is a step-by-step illustration (in Python/pandas) of how one can arrive at the conclusion that the “IF” (Internally Flawless) diamonds in this dataset end up having a comparatively low average total price—not because IF clarity is “cheap,” but because most IF stones here are quite small in carat weight or have other characteristics (e.g. lab, color) that bring their total ticket price down. In reality, IF diamonds often command a high price once you account for size, color, cut, and lab.

--------------------------------------------------------------------------------
1. Load the data and compute the average price by clarity
--------------------------------------------------------------------------------

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# -- Paste the raw data into a list of dictionaries --
data = [
    {"IDNO":1,"WEIGHT":0.3,"COLOR":"D","CLARITY":"VS2","RATER":"GIA","PRICE":1302},
    {"IDNO":2,"WEIGHT":0.3,"COLOR":"E","CLARITY":"VS1","

## Load and Display an Image from a URL

This code retrieves an image from a specified URL and displays it using the **PIL (Pillow) library**.

In [ ]:
from PIL import Image
import requests
from io import BytesIO

image_url = "https://miro.medium.com/v2/resize:fit:4800/format:webp/1*VPRpf0YnchAwN0mjeAz4pA.jpeg"
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img.show()

## Create an Image Analysis Prompt

The following code constructs a **structured prompt** for analyzing an image. It sends both **text input** and an **image URL** to an AI model for interpretation.


In [ ]:
image_prompt = [
                    {"type": "text", "text": 'what is wrong with this image?'},
                    {"type": "image_url", "image_url": {
                        "url": image_url}
                    }
                ]

In [ ]:
gpt_result = openai_gpt_help(prompt=image_prompt)

In [ ]:
display(Markdown(gpt_result))

In [ ]:
o1_result = openai_o_help(prompt=image_prompt)

In [ ]:
display(Markdown(o1_result))

## References  
- **OpenAI Reasoning Models Guide**: [OpenAI](https://platform.openai.com/docs/guides/reasoning)  
- **OpenAI Reasoning Models Best Practices Guide**: [OpenAI](https://platform.openai.com/docs/guides/reasoning-best-practices)  
- **Colin Jarvis. “Reasoning with O1.” DeepLearning.AI.** Accessed February 14, 2025. [DeepLearning.AI](https://www.deeplearning.ai/short-courses/reasoning-with-o1/)  